<a href="https://colab.research.google.com/github/Preetiirs/Preetiirs/blob/main/Langchain_LCEL.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Modern AI Pro from Balaji and Arvind
The goal of this notebook is to introduce you to various aspects of Langchain used with an opensource light weight model, Orca.

In [ ]:
!pip install --upgrade --quiet langchain langchain-experimental duckduckgo-search
!pip install --upgrade --quiet ctransformers ctransformers[cuda]

In [ ]:
import os
os.environ['XDG_CACHE_HOME'] = 'drive/MyDrive/llms/model/cache/'

from langchain.llms import CTransformers
llm = CTransformers(model='TheBloke/Orca-2-13B-GGUF',
                    model_type="llama",
                    config = {'gpu_layers':50},
                    device = 0,
                   )

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Fetching 1 files:   0%|          | 0/1 [00:00<?, ?it/s]

config.json:   0%|          | 0.00/29.0 [00:00<?, ?B/s]

Fetching 1 files:   0%|          | 0/1 [00:00<?, ?it/s]

orca-2-13b.Q2_K.gguf:   0%|          | 0.00/5.43G [00:00<?, ?B/s]

In [ ]:
from langchain_core.prompts import ChatPromptTemplate
prompt = ChatPromptTemplate.from_template("tell me a joke about {foo}")
chain = prompt | llm
chain.invoke({"foo": "bears"})

".\nAgent: Sure, here's one. Why did the bear break up with his boyfriend?"

# Writing & executing code with LLMs
We will both write code

In [ ]:
from langchain_core.output_parsers import StrOutputParser
from langchain_experimental.utilities import PythonREPL
def _sanitize_output(code):
    parts = code.split("```")
    python_code = parts[1].strip() if len(parts) > 1 else ""
    return python_code

In [ ]:
template = """Write python code to solve the user's problem: {problem}.

Return only python code in Markdown format, e.g.:

```python
....
```"""
prompt = ChatPromptTemplate.from_template(template)
chain = prompt | llm | _sanitize_output | PythonREPL().run
chain.invoke({"problem": "print the numbers 1 to 10"})

'1\n2\n3\n4\n5\n6\n7\n8\n9\n10\n'

#Search + LLMs

In [ ]:
from langchain.tools import DuckDuckGoSearchRun
search = DuckDuckGoSearchRun()
prompt = ChatPromptTemplate.from_template("""turn the following user input into a search query for a search engine:
{input}""")
chain = prompt | llm | StrOutputParser() | search

In [ ]:
chain.invoke({"input": "The current PM of India"})



"Category: History & Society In full: Narendra Damodardas Modi Born: September 17, 1950, Vadnagar, India (age 73) Title / Office: prime minister (2014-), India Political Affiliation: Bharatiya Janata Party Rashtriya Swayamsevak Sangh See all related content → Recent News Jan. 22, 2024, 10:59 AM ET (AP) I ndia's prime minister, Narendra Modi, has cut a confident figure in recent weeks. As his Bharatiya Janata party (BJP) swept three major state elections in December, Modi did not hold back... June 27, 2023, 6:01 AM PDT By Sakshi Venkatraman WASHINGTON — Prime Minister Narendra Modi's official state visit turned the nation's capital into a microcosm of Indian politics on Thursday. Narendra Modi, India's prime minister, inaugurated the temple on Monday, months before an election, even though it is only 70 percent finished. By Mujib Mashal and Hari Kumar Photographs by... Prime Minister Narendra Modi will help consecrate the unfinished temple on Monday, on the site of a 16th century mosque

#SQL + LLMs
Let's build it up a notch and go big. How to take existing data and combine that with LLMs.

In [ ]:
%load_ext sql

The sql extension is already loaded. To reload it, use:
  %reload_ext sql


In [ ]:
import pandas as pd
df_cfemotion = pd.read_csv('https://archive.org/download/crowdflower/text_emotion.csv', on_bad_lines='skip', encoding='latin-1')
df_cfemotion

,tweet_id,sentiment,author,content
0,1956967341,empty,xoshayzers,@tiffanylue i know i was listenin to bad habi...
1,1956967666,sadness,wannamama,Layin n bed with a headache ughhhh...waitin o...
2,1956967696,sadness,coolfunky,Funeral ceremony...gloomy friday...
3,1956967789,enthusiasm,czareaquino,wants to hang out with friends SOON!
4,1956968416,neutral,xkilljoyx,@dannycastillo We want to trade with someone w...
...,...,...,...,...
39995,1753918954,neutral,showMe_Heaven,@JohnLloydTaylor
39996,1753919001,love,drapeaux,Happy Mothers Day All my love
39997,1753919005,love,JenniRox,Happy Mother's Day to all the mommies out ther...
39998,1753919043,happiness,ipdaman1,@niariley WASSUP BEAUTIFUL!!! FOLLOW ME!! PEE...


In [ ]:
import sqlite3 as sql
conn = sql.connect('/content/cfemotion.db')
df_cfemotion.to_sql('tb_emotion', conn)

40000

In [ ]:
%%sql
sqlite:///cfemotion.db
select * from tb_emotion LIMIT 5;

Done.


index,tweet_id,sentiment,author,content
0,1956967341,empty,xoshayzers,@tiffanylue i know i was listenin to bad habit earlier and i started freakin at his part =[
1,1956967666,sadness,wannamama,Layin n bed with a headache ughhhh...waitin on your call...
2,1956967696,sadness,coolfunky,Funeral ceremony...gloomy friday...
3,1956967789,enthusiasm,czareaquino,wants to hang out with friends SOON!
4,1956968416,neutral,xkilljoyx,"@dannycastillo We want to trade with someone who has Houston tickets, but no one will."


In [ ]:
from langchain_community.utilities import SQLDatabase
from langchain_core.runnables import RunnablePassthrough
template = """Based on the table schema below, write a SQL query that would answer the user's question:
{schema}

Question: {question}
SQL Query:"""
prompt = ChatPromptTemplate.from_template(template)

In [ ]:
db = SQLDatabase.from_uri("sqlite:///cfemotion.db")
def get_schema(_):
    return db.get_table_info()
def run_query(query):
    return db.run(query)

In [ ]:
sql_response = (
    RunnablePassthrough.assign(schema=get_schema)
    | prompt
    | llm.bind(stop=["\nSQLResult:"])
    | StrOutputParser()
)

In [ ]:
sql_response.invoke({"question": "How many rows are there?"})

' SELECT COUNT(*) FROM tb_emotion;'

In [ ]:
sql_chain = (
    RunnablePassthrough.assign(query=sql_response).assign(
        schema=get_schema,
        response=lambda x: db.run(x["query"]),
    )
)

In [ ]:
sql_chain.invoke({"question": "How many rows are there?"})

{'question': 'How many rows are there?',
 'query': ' SELECT COUNT(*) FROM tb_emotion;',
 'schema': '\nCREATE TABLE tb_emotion (\n\t"index" INTEGER, \n\ttweet_id INTEGER, \n\tsentiment TEXT, \n\tauthor TEXT, \n\tcontent TEXT\n)\n\n/*\n3 rows from tb_emotion table:\nindex\ttweet_id\tsentiment\tauthor\tcontent\n0\t1956967341\tempty\txoshayzers\t@tiffanylue i know  i was listenin to bad habit earlier and i started freakin at his part =[\n1\t1956967666\tsadness\twannamama\tLayin n bed with a headache  ughhhh...waitin on your call...\n2\t1956967696\tsadness\tcoolfunky\tFuneral ceremony...gloomy friday...\n*/',
 'response': '[(40000,)]'}

In [ ]:
sql_chain.invoke({"question": "How many rows have a sentiment column is neutral"})

{'question': 'How many rows have a sentiment column is neutral',
 'query': " SELECT COUNT(*) FROM tb_emotion WHERE sentiment = 'neutral'",
 'schema': '\nCREATE TABLE tb_emotion (\n\t"index" INTEGER, \n\ttweet_id INTEGER, \n\tsentiment TEXT, \n\tauthor TEXT, \n\tcontent TEXT\n)\n\n/*\n3 rows from tb_emotion table:\nindex\ttweet_id\tsentiment\tauthor\tcontent\n0\t1956967341\tempty\txoshayzers\t@tiffanylue i know  i was listenin to bad habit earlier and i started freakin at his part =[\n1\t1956967666\tsadness\twannamama\tLayin n bed with a headache  ughhhh...waitin on your call...\n2\t1956967696\tsadness\tcoolfunky\tFuneral ceremony...gloomy friday...\n*/',
 'response': '[(8638,)]'}

In [ ]:
sql_chain.invoke({"question": "Print the content columns where the content has the text @dannycastillo"})


{'question': 'Print the content columns where the content has the text @dannycastillo',
 'query': " SELECT content FROM tb_emotion WHERE content LIKE '%@dannycastillo%'",
 'schema': '\nCREATE TABLE tb_emotion (\n\t"index" INTEGER, \n\ttweet_id INTEGER, \n\tsentiment TEXT, \n\tauthor TEXT, \n\tcontent TEXT\n)\n\n/*\n3 rows from tb_emotion table:\nindex\ttweet_id\tsentiment\tauthor\tcontent\n0\t1956967341\tempty\txoshayzers\t@tiffanylue i know  i was listenin to bad habit earlier and i started freakin at his part =[\n1\t1956967666\tsadness\twannamama\tLayin n bed with a headache  ughhhh...waitin on your call...\n2\t1956967696\tsadness\tcoolfunky\tFuneral ceremony...gloomy friday...\n*/',
 'response': "[('@dannycastillo We want to trade with someone who has Houston tickets, but no one will.',)]"}

In [ ]:
template = """Evaluate if this guy's tweet here is happy and sensible:
 {response}"""
prompt_response = ChatPromptTemplate.from_template(template)

In [ ]:
from langchain.callbacks.tracers import ConsoleCallbackHandler

full_chain = (
    RunnablePassthrough.assign(query=sql_response).assign(
        schema=get_schema,
        response=lambda x: db.run(x["query"]),
    )
    | prompt_response | llm
)
full_chain.invoke({"question": "Print the content columns where the content has the text @dannycastillo"})

"\n\nAI Assistant: I would say that the tweet seems to express a sense of frustration or disappointment rather than happiness, as the user (@dannycastillo) is unable to find someone willing to trade Houston tickets. However, it's not necessarily unreasonable, as it depends on the context and the urgency of the situation."

# Give your feedback in Menti/Discord
If you guys are in the bootcamp, tell us how well this worked in Discord.